# 🏛️ Temple Crowd Detection — YOLOv8 Training Pipeline (Google Colab)

**Full end-to-end pipeline: dataset → training → fine-tuning → evaluation → export → download**

| Step | What happens |
|------|-------------|
| 1 | Install packages & verify GPU (T4 / A100) |
| 2 | Download Roboflow dataset (Version 5 — largest & best labelled) |
| 3 | Dataset health check — class distribution, image count, label visualisation |
| 4 | Train **YOLOv8m** with advanced fine-tuning hyperparameters on GPU |
| 5 | Full evaluation — mAP@50, mAP@50-95, Precision, Recall, F1 |
| 6 | Visual inspection — Confusion Matrix, PR Curve, F1 Curve, training loss graphs |
| 7 | Real-life inference testing on unseen test images with bounding boxes |
| 8 | Export to **ONNX** (production) + **TorchScript** (edge devices) |
| 9 | Download `best.pt`, `best.onnx`, all evaluation plots to your PC |

---
> ✅ **Requirements:** Google Colab with **T4 GPU** enabled  
> `Runtime` → `Change runtime type` → `T4 GPU` → `Save`  
> Then: `Runtime` → `Run all`  
>  
> ⏱️ **Estimated training time:** 20–40 min depending on dataset size

In [ ]:
# ── Cell 1: Install packages & verify GPU ─────────────────────────────────────
!pip install -q ultralytics roboflow

import torch
from ultralytics import YOLO
import os, glob, shutil
from IPython.display import display, Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from pathlib import Path

print("=" * 55)
print("  ENVIRONMENT CHECK")
print("=" * 55)
print(f"  PyTorch version   : {torch.__version__}")
print(f"  CUDA available    : {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb  = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"  GPU               : {gpu_name}")
    print(f"  VRAM              : {vram_gb:.1f} GB")
    BATCH_SIZE = 16 if vram_gb >= 14 else 8   # auto-adjust for smaller GPUs
    print(f"  Auto batch size   : {BATCH_SIZE}")
else:
    print("  ❌ No GPU detected — go to Runtime > Change runtime type > T4 GPU")
    BATCH_SIZE = 4

print("=" * 55)
print("✅ Setup complete — proceed to Cell 2")

In [ ]:
# ── Cell 2: Download Roboflow Dataset (Version 5) ─────────────────────────────
from roboflow import Roboflow

rf      = Roboflow(api_key="wsdLXeUN3MmbPrvRMrqF")
project = rf.workspace("ricky-sambora").project("crowd-detection-7suou")
version = project.version(5)                         # Version 5 = largest dataset
dataset = version.download("yolov8")

# Store dataset root for all future cells
DATASET_DIR  = Path(dataset.location)
DATA_YAML    = DATASET_DIR / "data.yaml"

print("\n" + "=" * 55)
print("  DATASET INFO")
print("=" * 55)
print(f"  Location  : {DATASET_DIR}")
print(f"  YAML file : {DATA_YAML}")

# Count images per split
for split in ["train", "valid", "test"]:
    img_dir = DATASET_DIR / split / "images"
    lbl_dir = DATASET_DIR / split / "labels"
    imgs = list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png")) if img_dir.exists() else []
    lbls = list(lbl_dir.glob("*.txt")) if lbl_dir.exists() else []
    print(f"  {split:<7}: {len(imgs):>4} images  |  {len(lbls):>4} labels")

print("=" * 55)

# Print class names from data.yaml
import yaml
with open(DATA_YAML) as f:
    meta = yaml.safe_load(f)
print(f"\n  Classes ({meta.get('nc', '?')}): {meta.get('names', [])}")
print("\n✅ Dataset ready — proceed to Cell 3")

In [ ]:
# ── Cell 3: Dataset Health Check & Label Visualisation ────────────────────────
import matplotlib.patches as patches
import random, cv2
from PIL import Image as PILImage

# ── 3a. Class distribution across training labels ─────────────────────────────
label_files = list((DATASET_DIR / "train" / "labels").glob("*.txt"))
class_counts = {}
for lf in label_files:
    with open(lf) as f:
        for line in f:
            cls = int(line.split()[0])
            class_counts[cls] = class_counts.get(cls, 0) + 1

class_names = meta.get('names', [str(i) for i in range(meta.get('nc', 1))])

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
fig.suptitle('Dataset Health Check', fontsize=14, fontweight='bold')

# Bar chart — class distribution
labels  = [class_names[k] for k in sorted(class_counts)]
counts  = [class_counts[k] for k in sorted(class_counts)]
axes[0].bar(labels, counts, color='steelblue')
axes[0].set_title('Object Count per Class (Train split)')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=30)
for i, v in enumerate(counts):
    axes[0].text(i, v + 5, str(v), ha='center', fontsize=9)

# Image count per split
splits = ['train', 'valid', 'test']
split_counts = []
for s in splits:
    d = DATASET_DIR / s / "images"
    split_counts.append(len(list(d.glob("*.jpg")) + list(d.glob("*.png"))) if d.exists() else 0)
axes[1].bar(splits, split_counts, color=['steelblue', 'coral', 'mediumseagreen'])
axes[1].set_title('Images per Split')
axes[1].set_ylabel('Image Count')
for i, v in enumerate(split_counts):
    axes[1].text(i, v + 2, str(v), ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('dataset_health.png', dpi=150, bbox_inches='tight')
plt.show()

# ── 3b. Sample annotated images from training set ─────────────────────────────
train_imgs = list((DATASET_DIR / "train" / "images").glob("*.jpg"))
samples    = random.sample(train_imgs, min(6, len(train_imgs)))

fig2, ax2 = plt.subplots(2, 3, figsize=(15, 8))
fig2.suptitle('Sample Training Images with Ground-Truth Annotations', fontsize=13)
for ax, img_path in zip(ax2.flat, samples):
    img = plt.imread(str(img_path))
    h, w = img.shape[:2]
    ax.imshow(img)
    lbl_path = img_path.parent.parent / "labels" / (img_path.stem + ".txt")
    if lbl_path.exists():
        with open(lbl_path) as f:
            for line in f:
                parts = list(map(float, line.split()))
                cx, cy, bw, bh = parts[1]*w, parts[2]*h, parts[3]*w, parts[4]*h
                x1, y1 = cx - bw/2, cy - bh/2
                rect = patches.Rectangle((x1,y1), bw, bh, linewidth=2, edgecolor='lime', facecolor='none')
                ax.add_patch(rect)
    ax.axis('off')
    ax.set_title(img_path.name[:20], fontsize=8)
plt.tight_layout()
plt.savefig('sample_annotations.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Health check done — dataset_health.png & sample_annotations.png saved")

In [ ]:
# ── Cell 4: Train YOLOv8m with Advanced Fine-Tuning ───────────────────────────
# YOLOv8m = Medium variant — best accuracy/speed balance for crowd detection
# All hyperparameters are tuned specifically for dense crowd/people detection

model = YOLO('yolov8m.pt')   # starts from COCO-pretrained weights (transfer learning)

print("🚀 Starting training on GPU...")
print(f"   Model     : YOLOv8m (transfer learning from COCO)")
print(f"   Dataset   : {DATA_YAML}")
print(f"   Batch     : {BATCH_SIZE}")
print()

results = model.train(
    # ── Core settings ──────────────────────────────────────────────────────────
    data     = str(DATA_YAML),
    epochs   = 100,           # max epochs; early stopping will cut short if needed
    imgsz    = 640,           # standard YOLO input resolution
    batch    = BATCH_SIZE,    # auto-set to 16 (T4) or 8 (smaller GPUs)
    device   = 0,             # use GPU 0 (T4 on Colab)
    workers  = 2,             # data-loading threads

    # ── Regularisation & overfitting prevention ─────────────────────────────
    dropout  = 0.1,           # 10% dropout — prevents memorising one temple
    patience = 20,            # early stopping: halt if no gain for 20 epochs
    weight_decay = 0.0005,    # L2 regularisation

    # ── Learning rate scheduling (cosine) ────────────────────────────────────
    optimizer = 'AdamW',      # AdamW = best for fine-tuning pretrained models
    cos_lr    = True,         # cosine annealing — smooth LR decay to minimum
    lr0       = 0.001,        # initial LR (lower than default for fine-tuning)
    lrf       = 0.1,          # final LR = lr0 × lrf = 0.0001

    # ── Training augmentations (prevent overfitting on small datasets) ────────
    hsv_h    = 0.015,         # hue jitter (lighting variations)
    hsv_s    = 0.7,           # saturation jitter (outdoor/indoor scenes)
    hsv_v    = 0.4,           # brightness jitter (day/night/flash)
    fliplr   = 0.5,           # 50% horizontal flip
    mosaic   = 1.0,           # mosaic augmentation — combine 4 images
    mixup    = 0.1,           # MixUp blending — helps generalise
    copy_paste = 0.1,         # copy-paste crowd instances across images

    # ── Output ─────────────────────────────────────────────────────────────────
    plots    = True,          # auto-generate all evaluation charts
    save     = True,
    project  = 'runs/detect',
    name     = 'crowd_train',
    exist_ok = False,         # create a fresh numbered dir if run again
)

# Capture the exact save directory — NEVER hardcode 'runs/detect/crowd_train'
# because Ultralytics auto-increments (crowd_train2, crowd_train3 ...) on re-runs
TRAIN_DIR   = Path(results.save_dir)
WEIGHTS_DIR = TRAIN_DIR / 'weights'

print(f"\n✅ Training complete!")
print(f"📁 Results saved to : {TRAIN_DIR}")
print(f"🏆 Best weights     : {WEIGHTS_DIR / 'best.pt'}")

In [ ]:
# ── Cell 5: Full Model Evaluation (Validation Split) ──────────────────────────
from ultralytics import YOLO

best_pt    = WEIGHTS_DIR / 'best.pt'
best_model = YOLO(str(best_pt))

print(f"✅ Loaded: {best_pt}")
print()
best_model.info()   # print layer count, parameters, FLOPS

print("\n⏳ Running validation on held-out validation split...")
metrics = best_model.val(
    data   = str(DATA_YAML),
    imgsz  = 640,
    device = 0,
    plots  = True,   # saves confusion_matrix.png, PR_curve.png, F1_curve.png
    save_json = True # saves predictions in COCO JSON format
)

# ── Per-class results ──────────────────────────────────────────────────────────
print("\n" + "=" * 55)
print("  VALIDATION METRICS — per class")
print("=" * 55)
names = best_model.names
for i, name in names.items():
    try:
        ap50    = metrics.box.ap50[i]
        ap      = metrics.box.ap[i]
        print(f"  {name:<20}  mAP@50={ap50:.4f}  mAP@50-95={ap:.4f}")
    except Exception:
        pass

print()
print("=" * 55)
print("  OVERALL METRICS")
print("=" * 55)
print(f"  mAP@50       : {metrics.box.map50:.4f}")
print(f"  mAP@50-95    : {metrics.box.map:.4f}   ← PRIMARY metric")
print(f"  mAP@75       : {metrics.box.map75:.4f}")
print(f"  Precision    : {metrics.box.mp:.4f}")
print(f"  Recall       : {metrics.box.mr:.4f}")
f1 = 2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-9)
print(f"  F1 Score     : {f1:.4f}")
print("=" * 55)

# Quality gate
if metrics.box.map50 >= 0.80:
    print("🟢 EXCELLENT — mAP@50 ≥ 80%  (production ready)")
elif metrics.box.map50 >= 0.65:
    print("🟡 GOOD      — mAP@50 ≥ 65%  (usable, more data helps)")
else:
    print("🔴 LOW       — mAP@50 < 65%  (consider more epochs / data)")

In [ ]:
# ── Cell 6: Visual Evaluation Plots ───────────────────────────────────────────
# Display all auto-generated training and validation charts

plots_to_display = {
    'Training Loss & mAP Curves'  : 'results.png',
    'Confusion Matrix (normalised)': 'confusion_matrix_normalized.png',
    'Confusion Matrix (raw)'       : 'confusion_matrix.png',
    'Precision-Recall Curve'       : 'PR_curve.png',
    'F1-Confidence Curve'          : 'F1_curve.png',
    'Precision-Confidence Curve'   : 'P_curve.png',
    'Recall-Confidence Curve'      : 'R_curve.png',
    'Sample Training Batch'        : 'train_batch0.jpg',
    'Sample Validation Batch'      : 'val_batch0_pred.jpg',
}

print("📊 Loading evaluation charts from:", TRAIN_DIR)
shown = 0
for title, fname in plots_to_display.items():
    fpath = TRAIN_DIR / fname
    if fpath.exists():
        print(f"\n{'─'*50}")
        print(f"  {title}")
        print(f"{'─'*50}")
        display(Image(filename=str(fpath), width=800))
        shown += 1
    else:
        # try the val/ subdirectory where ultralytics saves some validation plots
        alt = TRAIN_DIR / 'val' / fname
        if alt.exists():
            print(f"\n  {title}")
            display(Image(filename=str(alt), width=800))
            shown += 1

print(f"\n✅ Displayed {shown} / {len(plots_to_display)} evaluation plots")

In [ ]:
# ── Cell 7: Real-Life Inference Testing on Unseen Images ──────────────────────
import random

# Source: test split → fallback to valid split if test doesn't exist
test_img_dir  = DATASET_DIR / 'test'  / 'images'
valid_img_dir = DATASET_DIR / 'valid' / 'images'

if test_img_dir.exists() and len(list(test_img_dir.glob('*.jpg'))) > 0:
    img_source = test_img_dir
    split_used = 'test'
else:
    img_source = valid_img_dir
    split_used = 'valid'
    print("⚠️  No test split found — using validation images for visual check")

all_imgs = list(img_source.glob('*.jpg')) + list(img_source.glob('*.png'))
sample   = random.sample(all_imgs, min(8, len(all_imgs)))   # pick 8 random images

print(f"🔍 Running inference on {len(sample)} unseen {split_used} images")
print(f"   Confidence threshold : 0.30")
print(f"   IoU threshold (NMS)  : 0.50\n")

preds = best_model.predict(
    source  = [str(p) for p in sample],
    imgsz   = 640,
    conf    = 0.30,       # 30% confidence — good balance; lower = more detections
    iou     = 0.50,       # NMS IoU threshold
    save    = True,       # save annotated images to disk
    verbose = False,
)

# Show annotated predictions
print("📸 Annotated predictions:\n")
for r in preds:
    out_path = Path(r.save_dir) / Path(r.path).name
    if not out_path.exists():
        # try globbing for the saved file
        matches = list(Path(r.save_dir).glob(Path(r.path).stem + '*'))
        out_path = matches[0] if matches else None
    n_boxes = len(r.boxes)
    crowd_density = "🟢 Low" if n_boxes < 5 else ("🟡 Medium" if n_boxes < 15 else "🔴 High")
    print(f"  {Path(r.path).name:<35}  Detections: {n_boxes:>3}  Crowd: {crowd_density}")
    if out_path and out_path.exists():
        display(Image(filename=str(out_path), width=600))

# ── Crowd density summary ─────────────────────────────────────────────────────
all_counts = [len(r.boxes) for r in preds]
print(f"\n📊 Inference Summary:")
print(f"  Images tested        : {len(preds)}")
print(f"  Avg detections/image : {np.mean(all_counts):.1f}")
print(f"  Max detections       : {max(all_counts)}")
print(f"  Min detections       : {min(all_counts)}")
print()
print("🎥 To test on a VIDEO, upload your .mp4 to Colab then run:")
print("   best_model.predict('your_video.mp4', save=True, conf=0.30, stream=True)")

In [ ]:
# ── Cell 8: Export to Production Formats ──────────────────────────────────────
import os

print("📦 Exporting trained model for production deployment...\n")

# ── ONNX export (for Python/Node.js backend — no PyTorch needed) ─────────────
print("1️⃣  Exporting to ONNX...")
onnx_path = best_model.export(
    format   = 'onnx',
    imgsz    = 640,
    dynamic  = True,      # variable batch size — required for live camera streams
    simplify = True,      # remove redundant ONNX nodes — smaller & faster
    opset    = 17,        # ONNX opset 17 = widest tool compatibility
)
ONNX_PATH = Path(str(onnx_path))
print(f"   ✅ ONNX saved  : {ONNX_PATH}")

# ── TorchScript export (for edge deployment → Raspberry Pi, Jetson Nano) ──────
print("\n2️⃣  Exporting to TorchScript...")
try:
    ts_path = best_model.export(format='torchscript', imgsz=640, optimize=True)
    TS_PATH = Path(str(ts_path))
    print(f"   ✅ TorchScript : {TS_PATH}")
except Exception as e:
    TS_PATH = None
    print(f"   ⚠️  TorchScript skipped: {e}")

# ── File size summary ──────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("  EXPORT SUMMARY")
print("=" * 50)

export_files = [
    (WEIGHTS_DIR / 'best.pt',  'PyTorch weights (primary)'),
    (WEIGHTS_DIR / 'last.pt',  'Last epoch weights (backup)'),
    (ONNX_PATH,                'ONNX (backend deployment)'),
]
if TS_PATH:
    export_files.append((TS_PATH, 'TorchScript (edge devices)'))

for fpath, desc in export_files:
    if fpath and Path(fpath).exists():
        size_mb = Path(fpath).stat().st_size / 1e6
        print(f"  ✅ {size_mb:>6.1f} MB  {desc}")
        print(f"           {fpath}")
    else:
        print(f"  ❌ Not found : {desc}")
print("=" * 50)

In [ ]:
# ── Cell 9: Full Training Summary ──────────────────────────────────────────────
print("=" * 60)
print("  🏛️  TEMPLE CROWD DETECTION — TRAINING COMPLETE")
print("=" * 60)

# Model performance
print(f"\n  📊 Model Performance:")
print(f"     mAP@50       : {metrics.box.map50:.4f}")
print(f"     mAP@50-95    : {metrics.box.map:.4f}")
print(f"     Precision    : {metrics.box.mp:.4f}")
print(f"     Recall       : {metrics.box.mr:.4f}")
f1 = 2 * metrics.box.mp * metrics.box.mr / (metrics.box.mp + metrics.box.mr + 1e-9)
print(f"     F1 Score     : {f1:.4f}")

# Training config
print(f"\n  ⚙️  Training Config:")
print(f"     Base model   : YOLOv8m (COCO pretrained)")
print(f"     Dataset      : Roboflow crowd-detection v5")
print(f"     Batch size   : {BATCH_SIZE}")
print(f"     Image size   : 640×640")
print(f"     Optimizer    : AdamW + Cosine LR")
print(f"     Dropout      : 10%")
print(f"     Augmentations: Mosaic, MixUp, HSV, Flip, CopyPaste")

# Saved files
print(f"\n  📁 Saved Files:")
all_save = [
    (WEIGHTS_DIR / 'best.pt',          'best.pt      — best checkpoint'),
    (WEIGHTS_DIR / 'last.pt',          'last.pt      — final epoch'),
    (ONNX_PATH,                        'best.onnx    — ONNX production'),
    (TRAIN_DIR / 'results.png',        'results.png  — loss/mAP curves'),
    (TRAIN_DIR / 'confusion_matrix_normalized.png', 'confusion_matrix_normalized.png'),
    (TRAIN_DIR / 'PR_curve.png',       'PR_curve.png'),
    (TRAIN_DIR / 'F1_curve.png',       'F1_curve.png'),
    ('dataset_health.png',             'dataset_health.png'),
    ('sample_annotations.png',         'sample_annotations.png'),
]
for fpath, label in all_save:
    exists = Path(str(fpath)).exists()
    mark   = '✅' if exists else '❌'
    size   = f"{Path(str(fpath)).stat().st_size/1e6:.1f} MB" if exists else "missing"
    print(f"     {mark} {label:<45} {size}")

print(f"\n  📂 Full train dir: {TRAIN_DIR}")
print("=" * 60)

In [ ]:
# ── Cell 10: Download All Trained Models & Plots to Your PC ────────────────────
# Each files.download() triggers a browser Save-As dialog (or saves to the
# VS Code Colab file explorer panel on the left).
from google.colab import files
import os

download_targets = [
    # Models (most important — copy these into your project)
    (str(WEIGHTS_DIR / 'best.pt'),   '🏆 best.pt         PyTorch weights (primary model)'),
    (str(ONNX_PATH),                 '⚡ best.onnx        ONNX (Node.js/Python backend)'),
    (str(WEIGHTS_DIR / 'last.pt'),   '💾 last.pt         Last epoch (backup)'),

    # Evaluation plots
    (str(TRAIN_DIR / 'results.png'),              '📈 results.png              Training curves'),
    (str(TRAIN_DIR / 'confusion_matrix_normalized.png'), '🔢 confusion_matrix_norm.png'),
    (str(TRAIN_DIR / 'PR_curve.png'),             '📉 PR_curve.png             Precision-Recall'),
    (str(TRAIN_DIR / 'F1_curve.png'),             '📊 F1_curve.png             F1 vs Confidence'),
    (str(TRAIN_DIR / 'val_batch0_pred.jpg'),      '🖼️  val_batch0_pred.jpg      Val predictions'),

    # Dataset analysis
    ('dataset_health.png',                        '📋 dataset_health.png       Class distribution'),
    ('sample_annotations.png',                    '🔍 sample_annotations.png   GT annotations'),
]

print("📥 Starting downloads...\n")
success, skipped = 0, 0
for fpath, description in download_targets:
    if os.path.exists(fpath):
        files.download(fpath)
        print(f"  ✅ {description}")
        success += 1
    else:
        print(f"  ⚠️  Skipped (not found): {os.path.basename(fpath)}")
        skipped += 1

print(f"\n{'='*60}")
print(f"  Downloaded : {success} files")
print(f"  Skipped    : {skipped} files (run missing cells first)")
print(f"{'='*60}")
print()
print("📁 WHERE TO PUT THE FILES IN YOUR PROJECT:")
print()
print("  best.pt   → ml-services/crowd-detection/models/best.pt")
print("  best.onnx → ml-services/crowd-detection/models/best.onnx")
print()
print("🔗 To use best.onnx in your Node.js backend:")
print("   const { InferenceSession } = require('onnxruntime-node')")
print("   const session = await InferenceSession.create('./models/best.onnx')")
print()
print("🎉 All done! Your crowd detection model is ready for production.")